In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, uniform
from torch import mm, exp, log, det, minimum, sqrt, mul
from torch.linalg import inv
import time
import pdb

In [8]:
def Gaussian(x, mu):
    '''
    Multivariate Gaussian distribution
    '''
    global Minv, detM
    return (1/(sqrt(detM*(2*np.pi)**D)))* \
            exp( -0.5 *(x-mu).T.mm(Minv).mm(x-mu))+log(detM)
    

def KE(x, mu, Minv=None):
    '''
    -log pi(p|q)
    '''
    global Minv, detM
    if(Minv):
        return mul(exp((x-mu).T.mm(Minv).mm((x-mu))) + log(detM), .5)
    else:
        mul(exp((x-mu).T.mm((x-mu))) + log(detM), .5)

def PE(x, mu):
    '''
    -log pi(q) where pi(q) is Gaussian 
    '''
    global Minv, detM
    return .5 * exp((x-mu).T.mm(Minv).mm(x-mu)) + log(detM)

In [9]:
def get_gradient(x, Mat, func):
    '''
    dH calculates K(p) or V(q) along with associated gradient.
    x: is a tensor
    Mat: is a mass matrix
    func: is the K or V function to calculate
    returns function output and gradient
    '''
    #pdb.set_trace()
    x.requires_grad_(True) # track math operations
    x.retain_grad() # keep gradient after backward()
    out = func(x, Mat)  # calculate given function
    out.backward(gradient=torch.ones(out.size()))  # Calculate grads
    x_grad = x.grad.data  # get gradients only
    x.grad = None  # reset gradient 
    x.requires_grad_(False)  # stop tracking
    return out.detach().clone(), x_grad 

# Unit Test
#t = torch.tensor([6., 4.]).view(2,1)
#tout, dt = dH(t, Minv, Tst2)

In [22]:
def stormer_verlet(V, q0, p0, Minv, epsilon, T):
    '''
    Stormer-Verlet should propagate a point in phase space
    (p,q) according to Hamiltonian H.
    
    dVdq: references a function yielding the derivative of
    potential energy V.
    q0: is initial position tensor
    p0: is initial momentum tensor
    epsilon: scalar time increment
    T: time interval length
    returns numpy arrays q and p
    '''
    q, p = q0, p0    
    for n in range(int(T//epsilon)):
        #pdb.set_trace()
        start = time.time()
        p -= epsilon * dH(q, Minv, V)/2. # half momentum step
        d1 = time.time()-start
        
        q += epsilon * p # full parameter step
        
        start = time.time()
        p -= epsilon * dH(q, Minv, V) # full momentum step
        d2 = time.time()-start
        
        print(f'n={n}  derivative time {d1:.2f} --- {d2:.2f}')
        
    return q,-p # Why wasn't this flipped?

In [23]:
def acceptance(q0,p0, q, p):
    r = min(1.,exp(KE(q0) + PE(p0) - KE(q) - PE(p)).item())
    obs = uniform.rvs()
    if obs < r:
        point = q
        acceptFlag = True
    else:
        point = q0
        acceptFlag = False
    return point, acceptFlag

In [24]:
D = 2
M = torch.tensor([[1., 0.8],
                  [0.8, 1.]])
Minv = inv(M)
detM = det(M).type(torch.float32)
mu = torch.tensor([[0.],[0.]],dtype=torch.float32)
N = 100

In [25]:
eps = 0.3
T = 9

Q0 = np.random.uniform(low=-4, high=4, size=(100,2))
# Loop iterations
for n in range(N):
    # sample a point from Q-space
    q0 = torch.from_numpy(Q0[n,:].reshape(2,1)).type(torch.float32)
    # sample p0
    p0 = torch.from_numpy(norm.rvs(size = 2))
    q, p = stormer_verlet(PE, q0, p0, Minv, eps, T)
    print(q)

TypeError: can't multiply sequence by non-int of type 'float'

In [26]:
x = torch.tensor([[.2],[.3]],requires_grad=True)
#g = 0.5 * exp((x-mu).T.mm(Minv).mm(x-mu)) + log(detM)